# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import time
import logging
import numpy as np
import pandas as pd

## Utils

In [2]:
def get_logger(level=logging.WARNING, force_level=False, name='logger'):
    
    logger_levels = [logging.CRITICAL, logging.ERROR, logging.WARNING,
                     logging.INFO, logging.DEBUG]
    
    if level not in logger_levels:
        raise ValueError('Invalid level name. Valid names {}'.format(logger_levels))
    
    formatter = 'UDACITY ARVATO PROJECT: %(asctime)s - %(levelname)s: %(message)s'
    date_format = '%d/%m/%Y %H:%M:%S'
    logging.basicConfig(format=formatter, datefmt=date_format, level=level)

    logger = logging.getLogger(name)
    if logger.getEffectiveLevel() == 0 or force_level:
        logger.setLevel(level)

    return logger


def set_log_level(verbose, debug):
    if verbose:
        get_logger(logging.INFO, force_level=True)
    if debug:
        get_logger(logging.DEBUG, force_level=True)


In [3]:
class Timer:
    
    def __init__(self, decimal_round=5):
        self.decimal_round = decimal_round

    def start(self):
        self.timer_start = time.perf_counter()
        self.timer_last = self.timer_start

    def lapse(self):
        last_lapse = self.timer_last
        self.timer_last = time.perf_counter()
        return round(self.timer_last - last_lapse, 5) 

    def total(self):
        return round(time.perf_counter() - self.timer_start, 5) 


In [4]:
def print_df_full(data):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    
    display(data)
    
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.max_colwidth', 50)


def convert_to_numeric(string):
    try: 
        return float(string)
    except ValueError:
        return string


## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [5]:
# load in the data
#azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
#customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

azdias = pd.read_csv(r'./datasets/azdias.csv', index_col='Unnamed: 0')
customers = pd.read_csv(r'./datasets/customers.csv', index_col='Unnamed: 0')

KeyboardInterrupt: 

In [ ]:
logger = get_logger()
timer = Timer(decimal_round=5)

### 0.1. Check dataset integrity

#### Verify that the datasets are as shown in the above description

In [ ]:
print('AZDIAS shape -', azdias.shape)
print('CUSTOMERS shape -', customers.shape)

In [ ]:
demographic_columns = azdias.columns
extra_columns = [col for col in customers.columns if col not in demographic_columns]

print('Extra columns:', extra_columns)

The dimensions and the extra columns, match the description.

In [ ]:
azdias.head()

In [ ]:
customers[demographic_columns].head()

In [ ]:
customers[extra_columns].head()

#### Solving WARNINGS

We can look at the datatypes of columns 18 and 19 since we got a warning while we loaded the data.

In [ ]:
warning_columns = azdias.columns[[18, 19]]
print('WARNING columns:', warning_columns)

In [ ]:
azdias[warning_columns].info()

In [ ]:
azdias['CAMEO_DEUG_2015'].unique()

In [ ]:
azdias['CAMEO_INTL_2015'].unique()

We have 'X' and 'XX' as values in these columns which have not been given in the description, also there are 'nan' values.

In [ ]:
def clean_warning_columns(df, columns, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    logger.info('Start cleaning warning columns')
    
    logger.debug('Replacing "X", "XX" and "nan" values with -1')
    replacements = {"X": np.nan, "XX": np.nan}
    df[columns] = df[columns].replace(replacements)
    logger.debug('Replaced columns: {0} in {1} seconds'.format(list(columns), timer.lapse()))
    
    logger.debug('Changing data type to float')
    df[columns] = df[columns].astype(float)
    logger.debug('Changed data type to float in {} seconds'.format(timer.lapse()))
    
    logger.info('End cleaning warning columns in {} seconds'.format(timer.total()))
    return df

In [ ]:
azdias = clean_warning_columns(azdias, warning_columns, verbose=True, debug=True)

In [ ]:
customers = clean_warning_columns(customers, warning_columns, verbose=True, debug=True)

Check whether the replacement has been carried out properly.

In [ ]:
azdias['CAMEO_DEUG_2015'].unique()

In [ ]:
azdias['CAMEO_INTL_2015'].unique()

### 0.2. Explore metadata datasets

Additionally we have been given two excel books.

- DIAS Attributes - Values 2017.xlsx - Has a description of demographic columns, information about range of values each column can take and their respective meanings.Loaded in metadata_values.
- DIAS Information Levels - Attributes 2017.xlsx - Has detailed information about each column with additional notes wherever required. Loaded in metadata_info_levels.

We can use these two dataframes to understand the data.

In [ ]:
metadata_values = pd.read_csv(r'./datasets/metadata/metadata_values.txt', sep='\t', encoding='UTF-16')
metadata_info_levels = pd.read_csv(r'./datasets/metadata/metadata_info_levels.txt', sep='\t', encoding='UTF-16')

In [ ]:
metadata_values.head(10)

In [ ]:
metadata_info_levels.head(10)

##### Metadata datasets preprocessing

We can use the ffill() method of pandas, to replace the NaN values of the dataset by the value of the previous row.

In [ ]:
metadata_values[['Attribute', 'Description']] = metadata_values[['Attribute', 'Description']].ffill()

In [ ]:
metadata_values.head(10)

##### Described features

We must check how many features are described in the metadata datasets.

In [ ]:
metadata_info_levels.drop_duplicates(subset=['Attribute'])

In [ ]:
metadata_values.drop_duplicates(subset=['Attribute'])

We can see that there are only 313 (in metadata_info_levels dataset) and 314 (in metadata_values dataset) different columns described, but in the AZDIAS dataset, there are 366 demographic columns. We need to check which features have no description in metadata datasets.

In [ ]:
metadata_columns = metadata_values.Attribute.unique().tolist()

described_columns = (set(metadata_columns) & set(demographic_columns))
undescribed_columns = (set(metadata_columns) - set(demographic_columns))

print("Number of described demographic columns: ", len(described_columns))
print("Number of undescribed demographic columns: ", len(undescribed_columns))

In [ ]:
print_df_full(metadata_values[['Attribute', 'Description']].drop_duplicates(subset=['Attribute']))

### 0.3. Data Preprocessing

##### Nulls normalization

In [ ]:
metadata_values.head(10)

Looking at the metadata of the columns in the provided excel workbooks, it can be seen that some columns have their own category to indicate a null value (for example categories like 'unknown', 'unknown / no main age detectable', 'numeric value', etc..). We could to unify the null values and replace these categories with np.nan values. Will only be done for the 272 columns that are described in the metadata datasets, as for the remaining 42 columns, we do not know the meaning of the values.

In [ ]:
def get_unkown_values_data(metadata, unkown_categories, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    logger.info('Start getting unkown categories from metadata')
    
    unknown_values = metadata[metadata['Meaning'].isin(unkown_categories)]
    unknown_values = unknown_values.reset_index(drop=True)
    logger.debug('Number of features with unknown categories:'.format(len(unknown_values)))
    
    logger.info('End getting unkown categories from metadata in {} seconds'.format(timer.total()))
    return unknown_values


In [ ]:
unknown_values = ['unknown', 'unknown / no main age detectable', 'numeric value', 'none']

unknown_data = get_unkown_values_data(metadata_values, unknown_values, verbose=True, debug=True)
print_df_full(unknown_data)

In [ ]:
def replace_unknown_values(data, metadata, unknown_values, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    unknown_data = get_unkown_values_data(metadata, unknown_values, verbose=verbose, debug=debug)
    
    logger.info('Start replacing unkown values')
    
    for attribute in unknown_data['Attribute']:
        
        if attribute in data.columns:
            unknown_categories = _get_categories(unknown_data, attribute)
            data[attribute] = data[attribute].apply(lambda x: np.nan if x in unknown_categories[attribute] else x)
            
            logger.debug('Replaced {0} to NaN in {1} column in {2} seconds'.format(
                unknown_categories[attribute], attribute, timer.lapse()))
        else:
            logger.debug('Attribute {} not in data'.format(attribute))
    
    logger.info('End replacing unkown values in {} seconds'.format(timer.total()))
    return data


def _get_categories(data, attribute):
    output = {}
    for attribute in data['Attribute']:
        categories = data[data['Attribute'] == attribute]['Value']
        categories = categories.astype(str).str.cat(sep=', ')
        categories = [str(x).strip() for x in categories.split(', ')]

        output[attribute] = [convert_to_numeric(x) for x in categories]
    return output


In [ ]:
azdias = replace_unknown_values(azdias, metadata_values, unknown_values, verbose=True, debug=True)

In [ ]:
customers = replace_unknown_values(customers, metadata_values, unknown_values, verbose=True, debug=True)

### 0.4. Data Univariate Analysis

#### Remove columns with a high percentage of Null values

In [ ]:
def get_missing_columns(data, threshold=0.3, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    logger.info('Start labelling columns as missing')
    
    data_missing_pctg = _get_missing_percentage_column(data)
    remove_columns = data_missing_pctg['column'][data_missing_pctg['missing_pctg'] > threshold].tolist()
    logger.debug('Columns labelled as missing: {}'.format(remove_columns))
        
    logger.info('End labelling columns as missing in {}'.format(timer.total()))
    return remove_columns
    

def _get_missing_percentage_column(data):
    missing_pctg = data.isna().sum() / len(data)
    
    return pd.DataFrame({'column': data.columns,
                         'missing_pctg': missing_pctg}
                       ).reset_index(drop=True)


In [ ]:
# missing columns in AZDIAS
missing_columns_azdias = get_missing_columns(azdias, verbose=True, debug=True)

In [ ]:
# missing columns in CUSTOMERS
missing_columns_customers = get_missing_columns(customers, verbose=True, debug=True)

In [ ]:
missing_columns = list(set(missing_columns_azdias) & set(missing_columns_customers))
missing_columns

In [ ]:
def drop_columns(data, column_list, verbose=False, debug=False):
    set_log_level(verbose, debug)
    timer.start()
    
    logger.info('Start removing columns')
    
    data = data.drop(column_list, axis=1)
    logger.debug('Removed columns: {}'.format(column_list))
    
    logger.info('End removing columns in {} seconds'.format(timer.total()))
    return data


In [ ]:
azdias = drop_columns(azdias, missing_columns, verbose=True, debug=True)

In [ ]:
customers = drop_columns(customers, missing_columns, verbose=True, debug=True)

In [ ]:
def plot_missing_percentage(data_1, data_2, min_percentage=0):
    df1 = _get_missing_percentage_column(data_1)
    df2 = _get_missing_percentage_column(data_2)
    
    fig = plt.figure(figsize=(10, 70))
    fig.suptitle("Missing Data percentages")

    axis = fig.add_subplot(121)
    axis.set_title("Azdias")
    sns.barplot(y="column", x="missing_pctg", 
                data=df1[df1['missing_pctg'] > min_percentage],
                ax=axis)

    axis = fig.add_subplot(122)
    axis.set_title("Customers")
    sns.barplot(y="column", x="missing_pctg", 
                data=df2[df2['missing_pctg'] > min_percentage],
                ax=axis)

    fig.tight_layout(rect=[0, 0.03, 1, 0.975])
    plt.show()

In [ ]:
plot_missing_percentage(azdias, customers)

#### Remove rows with a high percentage of Null values

#### Classify columns into Numerical or Categorical

#### Transforming Categorical columns into Numerical columns

#### Check ID columns

#### Imputing missing values

### 0.5. Feature Scaling

### 0.6. Build a Function

Build a function that encompasses the stages of **Preprocessing**, **Univariate Analysis** and **Feature Scaling** optionally, in order to apply the same data cleaning process to all datasets used in this project.

#### Check Preprocessing and Univariate Analysis functions

#### Save datasets after Preprocessing and Univariate Analysis steps

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

#### Load clean datasets

#### Scaling data

### 1.1. Principal Component Analysis

#### 1.1.1. Interpreting PCA components

### 1.2. K-Means Clustering

After having reduced the demographic dataset to principal components, we will use a K-Means model to segment the population.

#### 1.2.1. Select optimal K value with AZDIAS_PCA data

##### Choosing Number of Clusters

#### 1.2.2. Fit K-Means object with optimal K and AZDIAS PCA data

#### 1.2.3. Get AZDIAS PCA and CUSTOMERS PCA Clusters

##### AZDIAS Clusters

##### CUSTOMERS Clusters

##### Check results

#### 1.2.4. Get ratio between customers and general population for each Cluster

#### 1.2.5. Analyze Feature Weights in Clusters with highest percentage of customers

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
# mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

#### Check if target column is in dataset

#### Check number of events / no events

### 2.1. Selecting the evaluation metric

### 2.2. Apply Preprocessing and Univariate Analysis steps

#### 2.2.1. Split data for validation and train

### 2.3. Set a benchmark

### 2.4. Evaluate different binary classification estimators

### 2.5. Select the best binary classification estimator

#### 2.5.1. Give the best results after hyperparameter tunning

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
# mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')
# mailout_test = pd.read_csv(r'./datasets/mailout_test.csv', index_col='Unnamed: 0')